LINQ provides `two parallel architectures`: local queries for local object collections and interpreted queries for remote data sources.

***Local queries*** resolve to `query operators` in the `Enumerable` class

interpreted queries are `descriptive`(توصیفی). They operate over sequences that
implement `IQueryable<T>`, and they resolve to the query operators in the `Queryable` class

***interpreted queries*** use `expression trees` to translated `IQueryable` queries to for example `SQL queries`.

To `write` interpreted queries, you need to start with an `API` that e`xposes sequences of type IQueryable<T>`. Like `EFCore`.

It’s also possible to `generate an IQueryable<T>` wrapper around an ordinary `enumerable collection` by calling the `AsQueryable` method.

In [2]:
#r "nuget:Microsoft.EntityFrameworkCore"
#r "nuget:Microsoft.EntityFrameworkCore.SqlServer"

Installed Packages Microsoft.EntityFrameworkCore, 8.0.8 Microsoft.EntityFrameworkCore.SqlServer, 8.0.8

In [3]:
using System.Linq;
using Microsoft.EntityFrameworkCore;

public class Customer
{
    public int ID { get; set; }
    public string Name { get; set; }
}

public class NutshellContext : DbContext
{
    public virtual DbSet<Customer> Customers { get; set; }

    protected override void OnConfiguring (DbContextOptionsBuilder builder)
        => builder.UseSqlServer (@"Data Source=.; Initial Catalog=Nutshell;Trusted_Connection=True;TrustServerCertificate=True;");
        
    protected override void OnModelCreating (ModelBuilder modelBuilder)
        => modelBuilder
            .Entity<Customer>()
            .ToTable ("Persons")
            .HasKey (c => c.ID);
}

var dbContext = new NutshellContext();

IQueryable<string> query = from c in dbContext.Customers
//where c.Name.Contains ("a")
orderby c.Name.Length
select c.Name.ToUpper();

foreach(var item in query)
    Console.WriteLine(item);
    


علی
حسن
محمد حسین


In [ ]:
--EF Core translates this query into the following SQL:
SELECT UPPER([c].[Name])
FROM [Customers] AS [c]
WHERE CHARINDEX(N'a', [c].[Name]) > 0
ORDER BY CAST(LEN([c].[Name]) AS int)

### How Interpreted Queries Work

> ***First***, the compiler converts `query syntax` to `fluent syntax`.  
  
> ***Next***, the compiler resolves interpreted queries to query operators in the `Queryable` class `instead of` the `Enumerable` class

In [ ]:
public static IQueryable<TSource> Where<TSource> (this
IQueryable<TSource> source, Expression <Func<TSource,bool>> predicate)

> when `compiler` see ***Where***, choice use `Where` extension method in `Queryable`, beacuse `dbContext.Customers` is of type `DbSet<T>`, which implements `IQueryable<T>` (a subtype of IEnumerable<T>).

> An `expression tree` is an object model based on the types in `System.Linq.Expressions` that can be `inspected` at `runtime` (so that `EF Core` can later `translate` it to an SQL statement).

> `Queryable.Where` accepts a predicate wrapped in an `Expression<TDelegate>` type.

<div dir="rtl" style="width:90%; margin:auto; font-family:vazirmatn">
<p>قابلیت اصلی که باعث می‌شود از Expression Tree برای ترجمه کوئری‌های LINQ به SQL استفاده شود، توانایی آن در بازنمایی و تحلیل عبارات به شکل شیء است. به این معنا که شما به جای اجرای مستقیم کوئری، می‌توانید ساختار کوئری را به‌عنوان یک داده (object) ذخیره و پردازش کنید. سپس این ساختار می‌تواند به SQL یا هر زبان دیگری ترجمه شود.</p>
<p>ویژگی‌های مهم Expression Tree عبارتند از:</p>
<ol><li><p><strong>قابلیت بازنمایی کد به عنوان داده</strong>:
Expression Tree به شما امکان می‌دهد تا کدی مانند کوئری‌های LINQ را به‌عنوان یک ساختار داده (مثل یک درخت) ذخیره کنید. هر گره در این درخت بیان یک عملیات خاص (مانند فراخوانی متد، دسترسی به عضو، عملگرهای شرطی و ...) است.</p></li><li><p><strong>قابلیت تجزیه و تحلیل</strong>:
به دلیل اینکه کوئری‌های LINQ به صورت Expression Tree بازنمایی می‌شوند، می‌توان آن‌ها را تجزیه و تحلیل کرد. مثلاً شما می‌توانید بررسی کنید که درخت بیان شامل کدام عملیات‌ها است و بر اساس آن تصمیم بگیرید که چگونه این عملیات‌ها را به SQL یا زبان دیگری ترجمه کنید.</p></li><li><p><strong>تأخیر در اجرا (Deferred Execution)</strong>:
کوئری‌های LINQ تا زمانی که نیاز به نتیجه نداشته باشند، اجرا نمی‌شوند. درخت بیان، نمایشی از این کوئری را به‌صورت شیء حفظ می‌کند و تا زمانی که درخواست اجرا شود (مثل فراخوانی <code>ToList()</code> یا <code>First()</code>)، درخت بررسی و پردازش نمی‌شود. این به ORMها مثل Entity Framework اجازه می‌دهد تا ابتدا کوئری را بررسی کرده و آن را به SQL ترجمه کنند.</p></li><li><p><strong>قابلیت تغییر و بازنویسی</strong>:
Expression Tree قابل تغییر و بازنویسی است. این یعنی شما می‌توانید درخت بیان را در طول زمان تغییر دهید، بخش‌هایی از آن را اضافه یا حذف کنید و سپس آن را به SQL یا هر زبان دیگری ترجمه کنید.</p></li></ol>
<h3>مراحل استفاده از Expression Tree در ترجمه LINQ به SQL:</h3>
<ol><li><p><strong>ایجاد کوئری LINQ</strong>: وقتی یک کوئری LINQ نوشته می‌شود (مثلاً <code>from p in Products where p.Price &gt; 100 select p</code>)، این کوئری به جای اجرای فوری به یک Expression Tree تبدیل می‌شود.</p></li><li><p><strong>تجزیه درخت بیان</strong>: ORM (مثل Entity Framework) این درخت بیان را تجزیه می‌کند تا بفهمد چه عملیاتی قرار است انجام شود (مانند فیلترها، انتخاب فیلدها و ...).</p></li><li><p><strong>ترجمه به SQL</strong>: بر اساس این درخت بیان، کوئری SQL معادل ساخته می‌شود (مثلاً <code>SELECT * FROM Products WHERE Price &gt; 100</code>).</p></li><li><p><strong>اجرا و واکشی داده‌ها</strong>: کوئری SQL اجرا شده و داده‌ها از پایگاه داده واکشی می‌شوند.</p></li></ol>
</div>

### Execution

`Interpreted queries` follow a `deferred(به تعویق افتاده) execution` model. This means that the `SQL statement` is not generated `until` you `start enumerating` the query.

In [ ]:
//enumerating the same query twice results in the database being queried twice.

IQueryable<string> query = from c in dbContext.Customers
where c.Name.Contains ("a")
orderby c.Name.Length
select c.Name.ToUpper();

foreach (var item in query) // execute 1
    Console.WriteLine(item);

foreach (var item in query) //execute 2
    Console.WriteLine(item);

### Combining Interpreted and Local Queries

A query can include both `interpreted` and `local` operators.

A typical pattern is to have the ***local operators*** on the `outside` and the ***interpreted components*** on the `inside`;

In [ ]:
public static IEnumerable<string> Pair (this IEnumerable<string> source)
{
    string firstHalf = null;
    foreach (string element in source)
        if (firstHalf == null)
            firstHalf = element;
        else
        {
            yield return firstHalf + ", " + element;
            firstHalf = null;
        }
}

using(var dbContext = new NutshellContext ())
{
    IEnumerable<string> q = dbContext.Customers
        .Select (c => c.Name.ToUpper())
        .OrderBy (n => n)
        .Pair() // Local from this point on.
        .Select ((n, i) => "Pair " + i.ToString() + " = " + n);

    foreach (string element in q) Console.WriteLine (element);
}

In [ ]:
-- this query select of database

SELECT UPPER([c].[Name]) 
FROM [Customers] AS [c] 
ORDER BY UPPER([c].[Name])

--The remaining work is done locally.

### AsEnumerable

`Enumerable.AsEnumerable` is the ***simplest*** of all query operators.

In [ ]:
public static IEnumerable<TSource> AsEnumerable<TSource>
(this IEnumerable<TSource> source)
{
    return source;
}

Its purpose is to ***cast*** an `IQueryable<T>` sequence to `IEnumerable<T>`  
This causes the `remainder` of the query to `execute locally`.

In [ ]:
Regex wordCounter = new Regex (@"\b(\w|[-'])+\b");

using (var dbContext = new NutshellContext ())
{
    var query = dbContext.MedicalArticles
        .Where (article => article.Topic == "influenza" &&
                        wordCounter.Matches (article.Abstract).Count < 100);

    //SQL Server doesn’t support regular expressions, so EF Core
    //will throw an exception
}

In [ ]:
Regex wordCounter = new Regex (@"\b(\w|[-'])+\b");

using (var dbContext = new NutshellContext ())
{
    var query = dbContext.MedicalArticles
        .Where (article => article.Topic == "influenza")
        .AsEnumerable() // reminder query execute locally
        .Where (article => wordCounter.Matches (article.Abstract).Count < 100);
}

An alternative to calling `AsEnumerable` is to call `ToArray` or `ToList`. The `advantage of AsEnumerable` is  
- that it doesn’t force immediate query execution  

-  does it create any storage structure.